In [1]:
%%capture

import pandas as pd
import orca
import warnings; warnings.filterwarnings("ignore")

from lcog import datasources
from lcog import variables
from lcog import models

orca.run(['build_networks'])

import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline

from bokeh.io import output_notebook, show
from bokeh.plotting import Figure
from datashader.bokeh_ext import create_ramp_legend, create_categorical_legend

output_notebook()

import datashader.transfer_functions as tf

import datashader as ds
from datashader.colors import viridis

from ipywidgets import interact
from ipywidgets import FloatSlider
import ipywidgets as widgets

In [2]:

def proforma(far, weighted_rent, parcel_size, land_cost,
             parking_rate, sqft_per_rate, sqft_per_stall, per_sqft_parking_cost,
             loan_to_cost_ratio, drawdown_factor, interest_rate, loan_fees, cap_rate,
             building_efficiency, profit_factor, building_cost_per_sqft, construction_months,
             chart_components = False, multi_return = False):
    """
    Pure-python implementation of proforma logic.
    """
    # Building bulk
    building_sqft = parcel_size * far

    # Parking
    parking_stalls = building_sqft * parking_rate / sqft_per_rate
    park_sqft = parking_stalls * sqft_per_stall
    park_cost = park_sqft * per_sqft_parking_cost

    # Total space
    total_built_sqft = building_sqft + park_sqft
    parking_sqft_ratio = park_sqft / total_built_sqft

    # Construction costs
    build_cost = building_cost_per_sqft * building_sqft
    cost = build_cost + park_cost
    cost_sqft = (cost / total_built_sqft) * profit_factor
    building_cost = building_sqft * cost_sqft # cost to build the new building
    total_construction_costs = building_cost + land_cost # add cost to buy current building

    # Financing costs
    loan_amount = total_construction_costs * loan_to_cost_ratio
    interest = loan_amount * drawdown_factor * (interest_rate / 12 * construction_months)
    points = loan_amount * loan_fees
    total_financing_costs = interest + points
    total_development_costs = total_construction_costs + total_financing_costs

    # Revenue
    usable_space = building_sqft * (1 - parking_sqft_ratio) * building_efficiency
    building_revenue = usable_space * weighted_rent / cap_rate

    # Profit
    profit = building_revenue - total_development_costs
    
    if chart_components:
        components = pd.Series({'revenue':building_revenue, 'costs':total_development_costs,
                                'profit':profit})
        return components.plot.bar()
    
    if multi_return:
        return profit, building_revenue, total_development_costs
    
    return profit

def profitability_given_rent(weighted_rent):
    far = 4.0
    loan_to_cost_ratio = .7
    drawdown_factor = 0.6
    interest_rate = 0.05
    loan_fees = 0.02
    building_efficiency = 0.7
    cap_rate = 0.05
    profit_factor = 1.1

    # Parking-type specific
    parking_rate = 1.0
    sqft_per_rate = 1000.0
    sqft_per_stall = 250.0
    per_sqft_parking_cost = 81

    # Parcel-specific  info
    parcel_size = 30000.0
    land_cost = 2000000.0

    # Lookups
    building_cost_per_sqft = 158 ## This is a lookup (cost_per_sqft_for_this_height)
    construction_months = 18 ## A lookup of self.construction_months based on total_sqft

    ## Calculate profitability of a known example and confirm that result matches expectations
    result = proforma(far, weighted_rent, parcel_size, land_cost,
                     parking_rate, sqft_per_rate, sqft_per_stall, per_sqft_parking_cost,
                     loan_to_cost_ratio, drawdown_factor, interest_rate, loan_fees, cap_rate,
                     building_efficiency, profit_factor, building_cost_per_sqft, construction_months)
    return result


def proforma2(far, weighted_rent, parcel_size, land_cost,
             parking_rate, sqft_per_rate, sqft_per_stall, per_sqft_parking_cost,
             loan_to_cost_ratio, drawdown_factor, interest_rate, loan_fees, cap_rate,
             building_efficiency, profit_factor, parcel_coverage, height_per_story,
             chart_components = False, multi_return = False):
    """
    Pure-python implementation of proforma logic, with cost/height lookups built-in.
    """
    # Building bulk
    building_sqft = parcel_size * far
    stories = building_sqft / (parcel_size * parcel_coverage)  ## For underground parking
    height = stories * height_per_story  
    
    # Parking
    parking_stalls = building_sqft * parking_rate / sqft_per_rate
    park_sqft = parking_stalls * sqft_per_stall
    park_cost = park_sqft * per_sqft_parking_cost

    # Total space
    total_built_sqft = building_sqft + park_sqft
    parking_sqft_ratio = park_sqft / total_built_sqft

    # Construction costs
    if height <= 15:
        building_cost_per_sqft = 170  ##  These costs are for residential form
    elif height <= 55:
        building_cost_per_sqft = 190  # wood
    elif height <= 120:
        building_cost_per_sqft = 210  # concrete
    else:
        building_cost_per_sqft = 240  # steel

    build_cost = building_cost_per_sqft * building_sqft
    cost = build_cost + park_cost
    cost_sqft = (cost / total_built_sqft) * profit_factor
    building_cost = building_sqft * cost_sqft # cost to build the new building
    total_construction_costs = building_cost + land_cost # add cost to buy current building

    # Financing costs
    if total_built_sqft <= 10000:
        construction_months = 12
    elif total_built_sqft <= 20000:
        construction_months = 14
    elif total_built_sqft <= 50000:
        construction_months = 18
    else:
        construction_months = 24

    loan_amount = total_construction_costs * loan_to_cost_ratio
    interest = loan_amount * drawdown_factor * (interest_rate / 12 * construction_months)
    points = loan_amount * loan_fees
    total_financing_costs = interest + points
    total_development_costs = total_construction_costs + total_financing_costs

    # Revenue
    usable_space = building_sqft * (1 - parking_sqft_ratio) * building_efficiency
    building_revenue = usable_space * weighted_rent / cap_rate

    # Profit
    profit = building_revenue - total_development_costs
    
    if chart_components:
        components = pd.Series({'revenue':building_revenue, 'costs':total_development_costs,
                                'profit':profit})
        return components.plot.bar()
    
    if multi_return:
        return profit, building_revenue, total_development_costs
    
    return profit

## Proforma sensitivity tests

### Profitability as a function of annual rent/sqft

In [3]:
style = {'description_width': 'initial'}
interact(profitability_given_rent, weighted_rent=widgets.FloatSlider(min=0,max=50,step=.5,value=10, style=style));

interactive(children=(FloatSlider(value=10.0, description='weighted_rent', max=50.0, step=0.5, style=SliderSty…

### Profitiability given all proforma parameters

In [4]:
continuous_update=False
style = {'description_width': 'initial'}
interact(proforma, chart_components=False,
         far = widgets.FloatSlider(min=.1,max=50,step=.1,value=4, continuous_update=continuous_update, style=style),
         loan_to_cost_ratio = widgets.FloatSlider(min=0,max=1,step=.05,value=.7, continuous_update=continuous_update, style=style),
         drawdown_factor = widgets.FloatSlider(min=0,max=1,step=.05,value=.6, continuous_update=continuous_update, style=style),
         interest_rate = widgets.FloatSlider(min=0,max=.5,step=.005,value=.05, continuous_update=continuous_update, style=style),
         loan_fees = widgets.FloatSlider(min=0,max=1,step=.005,value=.02, continuous_update=continuous_update, style=style),
         building_efficiency = widgets.FloatSlider(min=0,max=1,step=.005,value=.7, continuous_update=continuous_update, style=style),
         cap_rate = widgets.FloatSlider(min=0.005,max=.3,step=.005,value=.05, continuous_update=continuous_update, style=style),
         profit_factor = widgets.FloatSlider(min=0,max=10,step=.1,value=1.1, continuous_update=continuous_update, style=style),
         parking_rate = widgets.FloatSlider(min=0,max=10,step=.1,value=1.0, continuous_update=continuous_update, style=style),
         sqft_per_rate = widgets.FloatSlider(min=0,max=10000,step=100,value=1000, continuous_update=continuous_update, style=style),
         sqft_per_stall = widgets.FloatSlider(min=0,max=5000,step=50,value=250, continuous_update=continuous_update, style=style),
         per_sqft_parking_cost = widgets.FloatSlider(min=0,max=1000,step=10,value=100, continuous_update=continuous_update, style=style),
         weighted_rent=widgets.FloatSlider(min=0,max=100,step=.5,value=40, continuous_update=continuous_update, style=style),
         parcel_size = widgets.FloatSlider(min=1000,max=100000,step=500,value=30000, continuous_update=continuous_update, style=style),
         land_cost = widgets.FloatSlider(min=0,max=10000000,step=100000,value=2000000, continuous_update=continuous_update, style=style),
         building_cost_per_sqft = widgets.FloatSlider(min=0,max=1000,step=5,value=195, continuous_update=continuous_update, style=style),
         construction_months = widgets.FloatSlider(min=1,max=100,step=1,value=18, continuous_update=continuous_update, style=style),
        );

interactive(children=(FloatSlider(value=4.0, continuous_update=False, description='far', max=50.0, min=0.1, st…

In [5]:
continuous_update=False
style = {'description_width': 'initial'}
interact(proforma2, chart_components=False,
         far = widgets.FloatSlider(min=.1,max=20,step=.2,value=4, continuous_update=continuous_update, style=style),
         loan_to_cost_ratio = widgets.FloatSlider(min=0,max=1,step=.05,value=.7, continuous_update=continuous_update, style=style),
         drawdown_factor = widgets.FloatSlider(min=0,max=1,step=.05,value=.6, continuous_update=continuous_update, style=style),
         interest_rate = widgets.FloatSlider(min=0,max=.5,step=.005,value=.05, continuous_update=continuous_update, style=style),
         loan_fees = widgets.FloatSlider(min=0,max=1,step=.005,value=.02, continuous_update=continuous_update, style=style),
         building_efficiency = widgets.FloatSlider(min=0,max=1,step=.005,value=.7, continuous_update=continuous_update, style=style),
         cap_rate = widgets.FloatSlider(min=0.005,max=.3,step=.005,value=.05, continuous_update=continuous_update, style=style),
         profit_factor = widgets.FloatSlider(min=0,max=10,step=.1,value=1.1, continuous_update=continuous_update, style=style),
         parking_rate = widgets.FloatSlider(min=0,max=10,step=.1,value=1.0, continuous_update=continuous_update, style=style),
         sqft_per_rate = widgets.FloatSlider(min=0,max=10000,step=100,value=1000, continuous_update=continuous_update, style=style),
         sqft_per_stall = widgets.FloatSlider(min=0,max=5000,step=50,value=250, continuous_update=continuous_update, style=style),
         per_sqft_parking_cost = widgets.FloatSlider(min=0,max=1000,step=10,value=100, continuous_update=continuous_update, style=style),
         weighted_rent=widgets.FloatSlider(min=0,max=100,step=.5,value=40, continuous_update=continuous_update, style=style),
         parcel_size = widgets.FloatSlider(min=1000,max=100000,step=500,value=30000, continuous_update=continuous_update, style=style),
         land_cost = widgets.FloatSlider(min=0,max=10000000,step=100000,value=2000000, continuous_update=continuous_update, style=style),
         parcel_coverage = widgets.FloatSlider(min=0,max=1,step=.005,value=.8, continuous_update=continuous_update, style=style),
         height_per_story = widgets.FloatSlider(min=8,max=25,step=1,value=12, continuous_update=continuous_update, style=style),
        );

interactive(children=(FloatSlider(value=4.0, continuous_update=False, description='far', max=20.0, min=0.1, st…